# Predicting Home Sale Price

This will attempt to be similar to the Zestimate used on Zillow, the Redfin estimate used on Redfin, or any of the valuations provided by Collateral Analytics, CoreLogic, or Quantarium on Realtor.com.

I will create a regression model and use it to predict valuation and compare to the other valuations on currently listed homes.

In [1]:
%run src/imports.py

In [2]:
sns.set(rc={'figure.figsize':(16,4)})

In [35]:
neighborhoods_cleaned = pd.read_pickle('data/neighborhoods_cleaned.pkl')
neighborhoods_cleaned.reset_index(inplace=True)
neighborhoods_cleaned.drop(['index','DAYS ON MARKET','PROPERTY TYPE','ADDRESS','CITY','STATE OR PROVINCE'],axis=1,inplace=True)
neighborhoods_cleaned = neighborhoods_cleaned.merge(pd.get_dummies(neighborhoods_cleaned['LOCATION'],drop_first=True), how='outer',left_index=True,right_index=True)
neighborhoods_cleaned.drop(['LOCATION'],axis=1,inplace=True)
neighborhoods_cleaned['SOLD DATE'] = pd.to_datetime(neighborhoods_cleaned['SOLD DATE'])
neighborhoods_cleaned['MONTH'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).month
neighborhoods_cleaned['YEAR'] = pd.DatetimeIndex(neighborhoods_cleaned['SOLD DATE']).year
neighborhoods_cleaned.drop(['SOLD DATE'],axis=1,inplace=True)

In [36]:
neighborhoods_cleaned

,ZIP OR POSTAL CODE,PRICE,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE,ROOMS PER SQFT * 1000,Alamo Heights,...,Willow Grove Sub (sc),Wilshire Terrace,Wilshire Village,Windcrest,Wolf Creek,Woodcrest,Woodlake,Woods Of Shavano,MONTH,YEAR
0,78257,345000,5662.0,2005.0,173.0,230.0,29.647801,-98.614186,2.011061,0,...,0,0,0,0,0,0,0,0,9,2020
1,78256,337500,9583.0,2004.0,156.0,54.0,29.650304,-98.629082,2.770083,0,...,0,0,0,0,0,0,0,0,6,2020
2,78256,375500,6605.0,2004.0,126.0,54.0,29.651842,-98.630650,2.189289,0,...,0,0,0,0,0,0,0,0,3,2021
3,78255,462000,9147.0,2014.0,170.0,95.0,29.617650,-98.643430,2.577320,0,...,0,0,0,0,0,0,0,0,10,2018
4,78256,624000,75794.0,2000.0,207.0,25.0,29.620646,-98.622295,1.826029,0,...,0,0,0,0,0,0,0,0,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20007,78222,153000,6969.0,1983.0,97.0,NaN,29.383410,-98.376520,3.170577,0,...,0,0,0,0,0,0,0,0,9,2019
20008,78222,189900,4791.0,2018.0,112.0,25.0,29.386700,-98.379900,3.523194,0,...,0,0,0,0,0,0,0,0,7,2018
20009,78222,128000,5662.0,1985.0,116.0,NaN,29.383092,-98.376562,4.537205,0,...,0,0,0,0,0,0,0,0,3,2019
20010,78222,196499,5662.0,2014.0,85.0,6.0,29.379346,-98.370269,3.012048,0,...,0,0,0,0,0,0,0,0,3,2019


In [40]:
neighborhoods_cleaned.dropna(inplace=True)

In [41]:
y = neighborhoods_cleaned['PRICE'].values
X = neighborhoods_cleaned.drop('PRICE',axis=1).values

## Building a Model

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [53]:
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
y_predict = rfr.predict(X_test)
print("score:", rfr.score(X_test, y_test))
mean_squared_error(y_test, y_predict)

score: 0.8689144515290705


1523379547.4565213

In [45]:
y_predict

array([207179.  , 231479.85, 178050.  , ..., 230713.83, 216602.12,
       277318.97])

In [46]:
y_test

array([205000, 250000, 176000, ..., 224500, 215899, 236680])